In [ ]:
#Carbon Credit Calculator - USES FLOW SENSOR

from datetime import datetime

class CarbonCreditCalculator:
    #Calculates credits using flow sensor data

    def __init__(self, fsme=1.2, alpha=0.01):
        self.fsme = fsme
        self.alpha = alpha

        self.cubex_price_inr = 20000
        self.eth_price_inr = 200000

        self.credit_history = []

    def calculate_E_actual(self, co2_ppm, gas_flow_L_per_min, time_hours=1.0):
        """Calculate actual emissions using FLOW SENSOR
        Formula: CO2 mass = CO2 concentration × Gas volume × CO2 density"""
        
        total_gas_volume_L = gas_flow_L_per_min * 60 * time_hours

        co2_fraction = co2_ppm / 1000000

        co2_volume_L = total_gas_volume_L * co2_fraction

        # Convert to mass (CO2 density ≈ 1.96 g/L at STP)
        co2_mass_g = co2_volume_L * 1.96

        co2_mass_tonnes = co2_mass_g / 1000000

        return co2_mass_tonnes

    def calculate_E_removed(self, weight_captured_g):
        """Physical CO2 removed by KOH
            2 KOH + CO2 → K2CO3 + H2O"""
        # Weight change is from K2CO3 formation
        # Approximate: 1g weight ≈ 0.32g CO2

        co2_captured_g = weight_captured_g * 0.32
        co2_captured_tonnes = co2_captured_g / 1000000

        return co2_captured_tonnes

    def calculate_E_net(self, co2_ppm, gas_flow, weight_captured_g, fvalid, baseline=100):
        """Net carbon credits
        E_net = (E_baseline - E_actual + E_removed) × Fvalid × Fsme × α"""
        
        # E_actual from FLOW SENSOR
        e_actual = self.calculate_E_actual(co2_ppm, gas_flow, time_hours=1.0)

        # E_removed from weight sensor
        e_removed = self.calculate_E_removed(weight_captured_g)

        # E_reduced
        e_baseline = baseline
        e_reduced = max(0, e_baseline - e_actual)

        # E_gross
        e_gross = e_reduced + e_removed

        # E_net formula
        e_net = e_gross * fvalid * self.fsme * self.alpha

        # Validation
        if fvalid < 0.95:
            e_net = 0.0
            status = "INVALID - Anomaly detected"
            is_valid = False
        else:
            status = "VALID - Credits awarded"
            is_valid = True

        results = {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'E_baseline': round(e_baseline, 6),
            'E_actual': round(e_actual, 6),
            'E_removed': round(e_removed, 6),
            'E_reduced': round(e_reduced, 6),
            'E_gross': round(e_gross, 6),
            'F_valid': round(fvalid, 4),
            'F_sme': self.fsme,
            'alpha': self.alpha,
            'E_net': round(e_net, 6),
            'carbon_credits': round(e_net, 6),
            'is_valid': is_valid,
            'status': status,
            'gas_flow_used': round(gas_flow, 2)
        }

        self.credit_history.append(results)
        return results

print("Credit Calculator loaded")